# Streamlit App UI Experimental

**This notebook demonstrates experimental features. The more stable streamlit app ui is demonstrated in `quickstart/dashboard_appui.ipynb`.**

This notebook demonstrates an app interface that runs alongside the dashboard.

In [ ]:
# %load_ext autoreload
# %autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
sys.path.append(str(Path().cwd().parent.parent.resolve()))

In [ ]:
from trulens.core import Tru
from trulens.core.utils.keys import check_keys

check_keys("OPENAI_API_KEY")

tru = Tru()
tru.reset_database()
tru.start_dashboard(force=True, _dev=Path().cwd().parent.parent.resolve())

## langchain example

In [ ]:
def load_langchain_app():
    # All relevant imports must be inside this function.

    from langchain.chains import ConversationChain
    from langchain.memory import ConversationSummaryBufferMemory
    from langchain_community.llms import OpenAI

    llm = OpenAI(temperature=0.9, max_tokens=128)

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=64,
        llm=llm,
    )

    # Conversational app puts it all together.
    app = ConversationChain(llm=llm, memory=memory)

    return app


app1 = load_langchain_app()

tru_app1 = tru.Chain(
    app1, app_id="langchain_app", initial_app_loader=load_langchain_app
)

## llama_index example

In [ ]:
from llama_index.readers.web import SimpleWebPageReader

# Be careful what you include as globals to be used by the loader function as it
# will have to be serialized. We enforce a size limit which prohibits large
# objects to be included in the loader's closure.

# This object will be serialized alongside `load_llamaindex_app` below.
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)


def load_llamaindex_app():
    from llama_index.core import VectorStoreIndex

    index = VectorStoreIndex.from_documents(documents)
    query_engine = index.as_query_engine()

    return query_engine


app2 = load_llamaindex_app()
tru_app2 = tru.Llama(
    app2, app_id="llamaindex_app", initial_app_loader=load_llamaindex_app
)

## basic app example

In [ ]:
from trulens.core.app import TruWrapperApp


def load_basic_app():
    def custom_application(prompt: str) -> str:
        return f"a useful response to {prompt}"

    return TruWrapperApp(custom_application)


app3 = load_basic_app()

tru_app3 = tru.Basic(
    app3, app_id="basic_app", initial_app_loader=load_basic_app
)

## custom app example

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import (
    CustomApp,  # our custom app
)


# Create custom app:
def load_custom_app():
    app = CustomApp()
    return app


app4 = load_custom_app()

# Create trulens wrapper:
tru_app4 = tru.Custom(
    app=app4,
    app_id="custom_app",
    # Make sure to specify using the bound method, bound to self=app.
    main_method=app4.respond_to_query,
    initial_app_loader=load_custom_app,
)

## Verification

You can get a list of apps that include the `initial_app_loader` with the following utility method.

In [ ]:
from trulens.core.schema import AppDefinition

for app_json in AppDefinition.get_loadable_apps():
    print(app_json["app_id"])